In [1]:
import json
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import pandas as pd
from sentence_transformers import SentenceTransformer

/home/MERHAWI/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('document_with_ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [4]:
es_client = Elasticsearch("http://localhost:9200")

In [5]:

index_settings = {
    "settings": {
        "number_of_shards" : 1,
        "number_of_replicas" : 0
    },
    "mappings" : {
        "properties" : {
            "text" : {"type" : "text"},
            "section" : {"type" : "text"},
            "question" : {"type" : "text"},
            "course" : {"type" : "keyword"},
            "id" : {"type" : "keyword"},
            "question_vector" : {
                "type" : "dense_vector",
                "dims" : 384,
                "index" : True,
                "similarity": "cosine"
            },
            "text_vector" : {
                "type" : "dense_vector",
                "dims" : 384,
                "index" : True,
                "similarity": "cosine"
            },
            "question_text_vector" : {
                "type" : "dense_vector",
                "dims" : 384,
                "index" : True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course_questions"
es_client.indices.delete(index = index_name, ignore_unavailable=True)
es_client.indices.create(index = index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [6]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)
    

100%|███████████████████████████████████████████████████████████████████████████████| 948/948 [01:04<00:00, 14.69it/s]


In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████| 948/948 [00:08<00:00, 112.97it/s]


In [8]:
query = "Can i still join the course?"

In [9]:
query_vec = model.encode(query)

In [10]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field" : field,
        "query_vector" : vector,
        "k" : 5,
        "num_candidates" : 10000,
        "filter" : {
            "term" : {
                "course" : course
                }
            }
        }
    search_query = {
        "knn" : knn,
        "_source" : ["text", "section", "question", "course", "id"]
    }
    
    es_result = es_client.search(
        index = index_name,
        body = search_query
    )
    result_docs = []
    
    for hit in es_result['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [11]:
def question_vector_knn(q):
    question = q["question"]
    course = q["course"]

    v_q = model.encode(question)
    return elastic_search_knn("question_vector", v_q, course)

In [12]:
df_ground_truth = pd.read_csv('ground_truth_data.csv')

In [13]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [14]:
def hit_rate(relavance_total):
    cnt = 0
    for line in relavance_total:
        if True in line:
            cnt = cnt + 1
    return cnt / len(relavance_total)

In [15]:
def mrr_score(relavance_total):
    score = 0.0
    for line in relavance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                score = score + 1 / (rank + 1)
        
    return score / len(relavance_total)

In [16]:
def evaluation_function(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        #results = search_function(query=q['question'], course=q['course'])
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)
    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr_score': mrr_score(relevance_total)
    }

In [17]:
evaluation_function(ground_truth, question_vector_knn)

100%|█████████████████████████████████████████████████████████████████████████████| 4735/4735 [01:41<00:00, 46.71it/s]


{'hit_rate': 0.737909186906019, 'mrr_score': 0.6191622668074627}

In [18]:
def text_vector_knn(q):
    question = q["question"]
    course = q["course"]

    v_q = model.encode(question)
    return elastic_search_knn("text_vector", v_q, course)

In [20]:
evaluation_function(ground_truth, text_vector_knn)

100%|██████████████████████████████████████████████████████████████████████████████| 4735/4735 [01:37<00:00, 48.53it/s]


{'hit_rate': 0.8270327349524815, 'mrr_score': 0.6902358324533618}

In [21]:
def question_text_vector_knn(q):
    question = q["question"]
    course = q["course"]

    v_q = model.encode(question)
    return elastic_search_knn("question_text_vector", v_q, course)

In [22]:
evaluation_function(ground_truth, question_text_vector_knn)

100%|██████████████████████████████████████████████████████████████████████████████| 4735/4735 [01:37<00:00, 48.69it/s]


{'hit_rate': 0.9102428722280888, 'mrr_score': 0.7961457233368543}

In [23]:
def elastic_search_knn_combined(vector,course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score" : {
                            "query" : {
                                "term" : {
                                    "course" : course
                                }
                            },
                            "script" : {
                                "source" : """
                                    cosineSimilarity(params.query_vector, "question_vector") +
                                    cosineSimilarity(params.query_vector, "text_vector") +
                                    cosineSimilarity(params.query_vector, "question_text_vector") + 1
                                """,
                                "params" : {
                                    "query_vector" : vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source" : ["text", "section", "question", "course", "id"]
    }

    es_result = es_client.search(
        index = index_name,
        body = search_query
    )
    result_docs = []
    
    for hit in es_result['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [24]:
def vector_combined_knn(q):
    question = q["question"]
    course = q["course"]

    v_q = model.encode(question)
    return elastic_search_knn_combined(v_q, course)

In [25]:
evaluation_function(ground_truth, vector_combined_knn)

100%|██████████████████████████████████████████████████████████████████████████████| 4735/4735 [01:37<00:00, 48.63it/s]


{'hit_rate': 0.8937697993664203, 'mrr_score': 0.7736923618444222}